# 1. TESTE DE WEB SCRAPING

## Importando bibliotecas necessárias e realizando Web Scraping

### Utilizando **Requests** para fazer requisições HTTP e baixar páginas web.
### Utilizando **beautifulsoup4** para extrair e manipular dados de HTML/XML de forma fácil.

In [2]:
import requests
from bs4 import BeautifulSoup
import os

url = "https://www.gov.br/ans/pt-br/acesso-a-informacao/participacao-da-sociedade/atualizacao-do-rol-de-procedimentos"
response = requests.get(url)

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36"
}


### Função para  download de arquivo PDF

In [3]:
def download_pdf(pdf_url, path="downloads"):
    os.makedirs(path, exist_ok=True)
    filename = os.path.join(path, pdf_url.split("/")[-1])

    response = requests.get(pdf_url, headers=headers, stream=True)
    if response.status_code == 200:
        with open(filename, "wb") as f:
            for chunk in response.iter_content(1024):
                f.write(chunk)
        print(f"Downloaded {filename}")
    else:
        print(f"Failed to download {pdf_url}")

### Verificando os links, para baixar os arquivos desejados

In [10]:
soup = BeautifulSoup(response.text, "html.parser")
pdf_links = [a["href"] for a in soup.find_all("a", href=True) if ".pdf" in a["href"]]
for link in pdf_links:
    print(f"Link found: {link}")

Link found: https://www.gov.br/ans/pt-br/arquivos/composicao/organograma_ans_geral_setembro_2024-1.pdf
Link found: https://www.gov.br/ans/pt-br/acesso-a-informacao/participacao-da-sociedade/atualizacao-do-rol-de-procedimentos/Anexo_I_Rol_2021RN_465.2021_RN627L.2024.pdf
Link found: https://www.gov.br/ans/pt-br/acesso-a-informacao/participacao-da-sociedade/atualizacao-do-rol-de-procedimentos/Anexo_II_DUT_2021_RN_465.2021_RN628.2025_RN629.2025.pdf
Link found: https://www.gov.br/ans/pt-br/arquivos/assuntos/consumidor/o-que-seu-plano-deve-cobrir/nota13_geas_ggras_dipro_17012013.pdf
Link found: https://www.gov.br/ans/pt-br/arquivos/composicao/organograma_ans_geral_setembro_2024-1.pdf


### Sabemos que o primeiro link da lista não é o que queremos, então ignoramos ele e baixamos apenas os próximos dois

In [8]:
response = requests.get(url, headers=headers)

if response.status_code == 200:
    soup = BeautifulSoup(response.text, "html.parser")
    pdf_links = [a["href"] for a in soup.find_all("a", href=True) if ".pdf" in a["href"]]

    pdf_links = pdf_links[1:]
    
    if len(pdf_links) > 1:
        for link in pdf_links[:2]:
            if link.startswith("/"):
                link = url + link
            download_pdf(link)
    else:
        print("No PDFs found")
else:
    print("Failed to fetch page")

Downloaded downloads/Anexo_I_Rol_2021RN_465.2021_RN627L.2024.pdf
Downloaded downloads/Anexo_II_DUT_2021_RN_465.2021_RN628.2025_RN629.2025.pdf


### Compactando os Arquivos em ZIP

In [9]:
import zipfile
import os

def compress_zip(path="downloads", name_compress_zip="anexos.zip"):
    path_zip = os.path.join(path, name_compress_zip)

    with zipfile.ZipFile(path_zip, "w", zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(path):
            for file in files:
                path_zip = os.path.join(root, file)
                zipf.write(path_zip, os.path.relpath(path_zip, path))
    print(f"Compressed {path_zip}")

compress_zip()
    

Compressed downloads/Anexo_I_Rol_2021RN_465.2021_RN627L.2024.pdf


# 2 . TESTE DE TRANSFORMAÇÃO DE DADOS

### Importando bibliotecas e definindo o path do pdf

In [14]:
import pdfplumber
import pandas as pd
import numpy as np
import os

pdf_path = "downloads/Anexo_I_Rol_2021RN_465.2021_RN627L.2024.pdf"

### Definindo as colunas esperadas da tabela, pra facilitar a organização.

In [15]:
expected_columns = [
    "PROCEDIMENTO", "RN(alteração)", "VIGÊNCIA", "OD", "AMB", "HCO", "HSO",
    "REF", "PAC", "DUT", "SUBGRUPO", "GRUPO", "CAPÍTULO"
]

### Definindo uma lista pra armazenar os dados estruturados, abrindo o pdf e percorrendo todas as páginas

In [18]:

structured_data = []

with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        tables = page.extract_tables()
        if not tables:
            continue
        for table in tables:
            table = [
                [item.replace("\n", " ").strip() if item else np.nan for item in line] 
                for line in table
            ]

            for line in table:
                if line == expected_columns:
                    continue

                if len(line) < len(expected_columns):
                    line += [np.nan] * (len(expected_columns) - len(line))  
                elif len(line) > len(expected_columns):
                    line = line[:len(expected_columns)]

                structured_data.append(line)

df = pd.DataFrame(structured_data, columns=expected_columns)

df.dropna(how="all", inplace=True)

df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

os.makedirs("csv_files", exist_ok=True)

df.to_csv("csv_files/tables_extracted_anexo1.csv", index=False, na_rep="")

df.head(15)

,PROCEDIMENTO,RN(alteração),VIGÊNCIA,OD,AMB,HCO,HSO,REF,PAC,DUT,SUBGRUPO,GRUPO,CAPÍTULO
0,PROCEDIMENTO,RN (alteração),VIGÊNCIA,OD,AMB,HCO,HSO,REF,PAC,DUT,SUBGRUPO,GRUPO,CAPÍTULO
1,ACONSELHAMENTO GENÉTICO,NaN,NaN,NaN,AMB,HCO,HSO,REF,NaN,NaN,"CONSULTAS, VISITAS HOSPITALARES OU ACOMPANHAME...",PROCEDIMENTOS GERAIS,PROCEDIMENTOS GERAIS
2,ATENDIMENTO/ACOMPANHAMENTO EM HOSPITAL-DIA PSI...,NaN,NaN,NaN,NaN,HCO,HSO,REF,NaN,109,"CONSULTAS, VISITAS HOSPITALARES OU ACOMPANHAME...",PROCEDIMENTOS GERAIS,PROCEDIMENTOS GERAIS
3,ATENDIMENTO INTEGRAL AO RECÉM-NASCIDO (SALA DE...,NaN,NaN,NaN,NaN,HCO,NaN,REF,NaN,NaN,"CONSULTAS, VISITAS HOSPITALARES OU ACOMPANHAME...",PROCEDIMENTOS GERAIS,PROCEDIMENTOS GERAIS
4,ATENDIMENTO MÉDICO DO INTENSIVISTA EM UTI GERA...,NaN,NaN,NaN,NaN,HCO,HSO,REF,NaN,NaN,"CONSULTAS, VISITAS HOSPITALARES OU ACOMPANHAME...",PROCEDIMENTOS GERAIS,PROCEDIMENTOS GERAIS
5,ATENDIMENTO PEDIÁTRICO A GESTANTES (3º TRIMESTRE),NaN,NaN,NaN,AMB,HCO,NaN,REF,NaN,NaN,"CONSULTAS, VISITAS HOSPITALARES OU ACOMPANHAME...",PROCEDIMENTOS GERAIS,PROCEDIMENTOS GERAIS
6,ATIVIDADE EDUCACIONAL PARA PLANEJAMENTO FAMILIAR,NaN,NaN,NaN,AMB,NaN,NaN,REF,NaN,NaN,"CONSULTAS, VISITAS HOSPITALARES OU ACOMPANHAME...",PROCEDIMENTOS GERAIS,PROCEDIMENTOS GERAIS
7,ATIVIDADE EDUCATIVA EM SAÚDE BUCAL,NaN,NaN,OD,NaN,NaN,NaN,NaN,NaN,NaN,"CONSULTAS, VISITAS HOSPITALARES OU ACOMPANHAME...",PROCEDIMENTOS GERAIS,PROCEDIMENTOS GERAIS
8,ATIVIDADE EDUCATIVA EM ODONTOLOGIA PARA PAIS E...,NaN,NaN,OD,NaN,NaN,NaN,NaN,NaN,NaN,"CONSULTAS, VISITAS HOSPITALARES OU ACOMPANHAME...",PROCEDIMENTOS GERAIS,PROCEDIMENTOS GERAIS
9,CONDICIONAMENTO EM ODONTOLOGIA (COM DIRETRIZ D...,NaN,NaN,OD,NaN,NaN,NaN,NaN,NaN,86,"CONSULTAS, VISITAS HOSPITALARES OU ACOMPANHAME...",PROCEDIMENTOS GERAIS,PROCEDIMENTOS GERAIS


In [19]:
df.shape

(3571, 13)